<a href="https://colab.research.google.com/github/balsrini/Hackathon2019/blob/master/Second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.utils import to_categorical
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM, concatenate , Input, Reshape, Dense, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
import numpy as np


Using TensorFlow backend.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
#all_filenames = listdir('gdrive/hackathon2019/images/')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['86.jpg', '87.jpg', '88.jpg', '89.jpg', '90.jpg']


In [14]:

images = []
all_filenames = listdir('/content/gdrive/My Drive/hackathon2019/images')
print(all_filenames)
all_filenames.sort()
for filename in all_filenames:      
    images.append(img_to_array(load_img('/content/gdrive/My Drive/hackathon2019/images/' + filename, target_size=(299, 299))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

# Run the images through inception-resnet and extract the features without the classification layer
IR2 = InceptionResNetV2(weights='imagenet', include_top=False)
features = IR2.predict(images)


['86.jpg', '87.jpg', '88.jpg', '89.jpg', '90.jpg']


W0724 12:11:43.215019 140486089746304 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 12:11:43.257253 140486089746304 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 12:11:43.270066 140486089746304 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0724 12:11:43.333410 140486089746304 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0724 12:11:43.335178 1404860897

219062272/219055592 [==============================] - 2s 0us/step


In [16]:
# We will cap each input sequence to 100 tokens
max_caption_len = 100
# Initialize the function that will create our vocabulary 
tokenizer = Tokenizer(filters='', split=" ", lower=False)

# Read a document and return a string
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# Load all the HTML files
X = []
all_filenames = listdir('/content/gdrive/My Drive/hackathon2019/html/')
all_filenames.sort()
print(all_filenames)
for filename in all_filenames:
    X.append(load_doc('/content/gdrive/My Drive/hackathon2019/html/'+filename))

# Create the vocabulary from the html files
tokenizer.fit_on_texts(X)

# Add +1 to leave space for empty words
vocab_size = len(tokenizer.word_index) + 1
# Translate each word in text file to the matching vocabulary index
sequences = tokenizer.texts_to_sequences(X)
# The longest HTML file
max_length = max(len(s) for s in sequences)

# Intialize our final input to the model
X, y, image_data = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        # Add the entire sequence to the input and only keep the next word for the output
        in_seq, out_seq = seq[:i], seq[i]
        # If the sentence is shorter than max_length, fill it up with empty words
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        # Map the output to one-hot encoding
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        # Add and image corresponding to the HTML file
        image_data.append(features[img_no])
        # Cut the input sentence to 100 tokens, and add it to the input data
        X.append(in_seq[-100:])
        y.append(out_seq)

X, y, image_data = np.array(X), np.array(y), np.array(image_data)


['86.html', '87.html', '88.html', '89.html', '90.html']


In [4]:
images = []
all_filenames = listdir('/content/gdrive/My Drive/hackathon2019/images/')
all_filenames.sort()
for filename in all_filenames:
    print(filename)
    images.append(img_to_array(load_img('/content/gdrive/My Drive/hackathon2019/images/'+filename, target_size=(299, 299))))
images = np.array(images, dtype=float)
images = preprocess_input(images)

IR2 = InceptionResNetV2(weights=None, include_top=False, pooling='avg')
#IR2.load_weights('/data/models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
features = IR2.predict(images)





max_caption_len = 100
tokenizer = Tokenizer(filters='', split=" ", lower=False)

def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

X = []
all_filenames = listdir('/content/gdrive/My Drive/hackathon2019/html/')
all_filenames.sort()
for filename in all_filenames:
    X.append(load_doc('/content/gdrive/My Drive/hackathon2019/html/'+filename))

tokenizer.fit_on_texts(X)

vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(X)
max_length = max(len(s) for s in sequences)
 
X, y, image_data = list(), list(), list()
for img_no, seq in enumerate(sequences):
    for i in range(1, len(seq)):
        in_seq, out_seq = seq[:i], seq[i]
        in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
        out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
        image_data.append(features[img_no])
        X.append(in_seq[-100:])
        y.append(out_seq)

X, y, image_data = np.array(X), np.array(y), np.array(image_data)

W0724 13:16:34.510891 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0724 13:16:34.545394 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0724 13:16:34.556464 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



86.jpg
87.jpg
88.jpg
89.jpg
90.jpg


W0724 13:16:34.619576 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0724 13:16:34.621147 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0724 13:16:34.925717 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0724 13:16:35.369212 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0724 13:16:36.132690 13965253

In [5]:
image_features = Input(shape=(1536,))
image_flat = Dense(128, activation='relu')(image_features)
ir2_out = RepeatVector(max_caption_len)(image_flat)

language_input = Input(shape=(max_caption_len,))
language_model = Embedding(vocab_size, 200, input_length=max_caption_len)(language_input)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = LSTM(256, return_sequences=True)(language_model)
language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)

decoder = concatenate([ir2_out, language_model])
decoder = LSTM(512, return_sequences=True)(decoder)
decoder = LSTM(512, return_sequences=False)(decoder)
decoder_output = Dense(vocab_size, activation='softmax')(decoder)

model = Model(inputs=[image_features, language_input], outputs=decoder_output)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')




# Create the encoder
#image_features = Input(shape=(8, 8, 1536,))
#image_flat = Flatten()(image_features)
#image_flat = Dense(128, activation='relu')(image_flat)
#ir2_out = RepeatVector(max_caption_len)(image_flat)

#language_input = Input(shape=(max_caption_len,))
#language_model = Embedding(vocab_size, 200, input_length=max_caption_len)(language_input)
#language_model = LSTM(256, return_sequences=True)(language_model)
#language_model = LSTM(256, return_sequences=True)(language_model)
#language_model = TimeDistributed(Dense(128, activation='relu'))(language_model)

# Create the decoder
#decoder = concatenate([ir2_out, language_model])
#decoder = LSTM(512, return_sequences=False)(decoder)
#decoder_output = Dense(vocab_size, activation='softmax')(decoder)

# Compile the model
#model = Model(inputs=[image_features, language_input], outputs=decoder_output)
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


W0724 13:17:21.108511 139652538976128 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [6]:
# Train the neural network

print(image_data.shape)
print(X.shape)
print(y.shape)
print(features.shape)

#model.summary()

from keras.callbacks import ModelCheckpoint

#Train model
filepath="/content/gdrive/My Drive/hackathon2019/org-weights-epoch-1000---loss-0.0000.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_weights_only=True, period=50)
callbacks_list = [checkpoint]

model.fit([image_data, X], y, batch_size=1000, callbacks=callbacks_list, shuffle=False, epochs=1000)

#model.fit([image_data, X], y, batch_size=64, shuffle=False, epochs=2)

W0724 13:17:41.331499 139652538976128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


(2306, 1536)
(2306, 100)
(2306, 436)
(5, 1536)
Epoch 1/1000
2306/2306 [==============================] - 154s 67ms/step - loss: 6.1726
Epoch 2/1000
2306/2306 [==============================] - 155s 67ms/step - loss: 6.0474
Epoch 3/1000
2306/2306 [==============================] - 154s 67ms/step - loss: 5.7893
Epoch 4/1000
2306/2306 [==============================] - 152s 66ms/step - loss: 5.7668
Epoch 5/1000


KeyboardInterrupt: ignored

In [0]:

# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None



In [0]:

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
    in_text = 'START'
    # iterate over the whole length of the sequence
    for i in range(900):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0][-100:]
        # pad input
        sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
        yhat = model.predict([photo,sequence], verbose=0)
       # print(yhat)
        # convert probability to integer
        yhat = np.argmax(yhat)
        # map integer to word
        word = word_for_id(yhat, tokenizer)
        #print(word)
        # stop if we cannot map the word
        if word is None:
            break
        # append as input for generating the next word
        in_text += ' ' + word
        # Print the prediction
        print(' ' + word, end='')
        # stop if we predict the end of the sequence
        if word == 'END':
            break
    return


In [9]:


# Load and image, preprocess it for IR2, extract features and generate the HTML
test_image = img_to_array(load_img('/content/gdrive/My Drive/hackathon2019/images/87.jpg', target_size=(299, 299)))
print(test_image.shape)
model.load_weights('/content/gdrive/My Drive/hackathon2019/org-weights-epoch-1000---loss-0.0000.hdf5')
test_image = np.array(test_image, dtype=float)
test_image = preprocess_input(test_image)
test_features = IR2.predict(np.array([test_image]))
generate_desc(model, tokenizer, np.array(test_features), 100)


(299, 299, 3)
 <!DOCTYPE html>
<html lang="en" dir="ltr">
<head>
<title>Basic 86</title>
<meta charset="iso-8859-1">
<link rel="stylesheet" href="styles/layout.css" type="text/css">
<!--[if lt IE 9]><script src="scripts/html5shiv.js"></script><![endif]-->
</head>
<body>
<div class="wrapper row1">
 <header id="header" class="clear">
 <div id="hgroup">
 <h1><a href="#">Basic 86</a></h1>
 <h2>Free HTML5 Website Template</h2>
 </div>
 <nav>
 <ul>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li><a href="#">Text Link</a></li>
 <li class="last"><a href="#">Text Link</a></li>
 </ul>
 </nav>
 </header>
</div>
<!-- content -->
<div class="wrapper row2">
 <div id="container" class="clear">
 <!-- Slider -->
 <section id="slider" class="clear">
 <figure><img src="images/demo/630x300.gif" alt="">
 <figcaption>
 <h2>Eu justo augue estas</h2>
 <p>Nullamlacus dui ipsum conseque loborttis non euisque morbi penas dapibulum orna. Urnaultrice